In [113]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle
from bs4 import BeautifulSoup
import time

### URL 이용 프레임 소스 다운로드

In [114]:
chrome_driver = '/Users/woon/Desktop/chromedriver'
driver = webdriver.Chrome(chrome_driver)
url = 'http://kipris.or.kr/kdtj/remocon/frame.jsp?kind=2&start=DR&userid=&menuKind=9&rtField&rights=TM'

driver.implicitly_wait(5)
driver.get(url)

driver.switch_to.frame(driver.find_element_by_name("right"))
src = driver.page_source
soup = BeautifulSoup(src)

driver.close()
soup

<html lang="ko"><head>
<title>도형코드(비엔나) 조회  - &lt; 상표분류</title>
<link href="/kdtj/remocon/css/common.css" rel="stylesheet"/>
<link href="/kdtj/remocon/css/popup_common.css" rel="stylesheet"/>
<script src="/kdtj/remocon/js/COCO5000.js"></script>
<script>

function goSearch(arg) {
    var searchText=document.getElementsByName("search_text")[0].value;
    var numberCheck="";
    
    searchText= searchText.replace("(","");
    searchText = searchText.replace(")","");
    
    if(arg == "") {
        if(searchText == "")
            document.search_form.section.value = arg;
        else{
            if( 2 <= searchText.length && searchText.length <= 6 ){
                for(var i=0;i<searchText.length;i++){
                    if('0' <= searchText.charAt(i) && searchText.charAt(i) <= '9')
                    numberCheck="Y";
                }
            }
            if(numberCheck == "Y")
                document.search_form.searchText.value = makeExpression('DB', searchText, ' LIKE ', '

### 대분류 딕트 pickle 파일 생성

In [115]:
l_category = []
for i in range(len(soup.select('a'))):
    l_category.append(soup.select('a')[i].get_text())


In [116]:
del l_category[0:2]
del l_category[-1]
del l_category[-1]

In [117]:
l_category
large_number = []
large_text = []
for i in range(len(l_category)):
    if(i%2==1):
        large_text.append(l_category[i])
    else:
        large_number.append(l_category[i])

In [118]:
large_zip = zip(large_number,large_text)
main_category = dict(large_zip)

main_category

{'01': '천체, 자연현상, 지도',
 '02': '사람\n\n[Note]\n(a) 사람을 표현하는 문자(27-03-01)\n(b) 머리는(02-1), (02-03), (02-05), (02-07)로 분류할수 있으나, (02-09-25)로 분류할 수 없음',
 '03': '동물\n\n[Note]\n(a) 네발짐승의 머리는(03-06-25)로 분류되지 않고 (03-01~03-05)\n사이에 분류됨\n(b) 동물을 표시하는 서체 및 \n활자서체(27-03-03)',
 '04': '초자연적·전설적·공상적 또는 미확인 존재\n\n[Note]\n문자와 도형으로 구성되는 사람, 상상의 동물(27-03)은 포함하지 않음',
 '05': '식물',
 '06': '풍경',
 '07': '건축물, 광고용 구조물, 문, 장벽',
 '08': '식품류',
 '09': '직물, 의류, 재봉부속물, 모자류, \n신발류',
 '10': '담배, 흡연 기구, 성냥, 여행용품, 부채, 세면 물품',
 '11': '가사 용품\n\n[Note]\n12, 13 또는 19류로 분류되는 가사 용품은 포함하지 않음',
 '12': '가구, 위생용 설비',
 '13': '조명기기, 무선 밸브, 가열, 조리 또는 냉장 기구, 세탁 기계, 건조 기구',
 '14': '철물용구, 연장, 사다리',
 '15': '기계류, 모터, 엔진',
 '16': '통신·녹음 또는 재생·컴퓨터·사진 촬영·영화 촬영·광학기계기구',
 '17': '시계 장치, 보석, 체중계 및 측정 기구',
 '18': '수송장비, 동물용 장비',
 '19': '포장용 용기, 각종 잡다한 제품',
 '20': '필기, 제도 또는 그림용 재료, 사무용품, 문방구 및 서적용품 ',
 '21': '게임, 장난감, 스포츠용품, 회전목마',
 '22': '악기 및 그 부속품, 음악 부속품, 종, 그림, 조각',
 '23': '무기, 탄약, 갑옷\n\n[Note]\n탱크(18-01-14), 군사 항공기(18-05-01), 및 군함(18-03)은 포함

### 중분류 딕트 pickle 파일 생성

In [119]:
medium_value = []
for i in range(len(soup.find_all('td', colspan='2'))):
    medium_value.append(soup.find_all('td', colspan='2')[i].get_text())

In [127]:
len(medium_value)

146

In [121]:
m_number = []
for i in range(len(soup.select('td.line'))):
    if soup.select('td.line')[i].get_text() !='':
        m_number.append(soup.select('td.line')[i].get_text())

In [128]:
m_number

['01',
 '03',
 '05',
 '07',
 '11',
 '13',
 '15',
 '17',
 '01',
 '03',
 '05',
 '07',
 '09',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '09',
 '11',
 '13',
 '15',
 '17',
 '01',
 '02',
 '03',
 '05',
 '07',
 '01',
 '03',
 '05',
 '07',
 '09',
 '11',
 '13',
 '01',
 '03',
 '06',
 '07',
 '19',
 '01',
 '03',
 '05',
 '11',
 '15',
 '01',
 '03',
 '05',
 '07',
 '01',
 '03',
 '05',
 '07',
 '09',
 '01',
 '03',
 '05',
 '01',
 '03',
 '07',
 '01',
 '03',
 '01',
 '03',
 '01',
 '03',
 '05',
 '07',
 '09',
 '11',
 '01',
 '03',
 '05',
 '07',
 '09',
 '01',
 '03',
 '01',
 '02',
 '03',
 '05',
 '01',
 '02',
 '03',
 '04',
 '05',
 '07',
 '01',
 '03',
 '07',
 '08',
 '09',
 '10',
 '11',
 '13',
 '19',
 '01',
 '05',
 '07',
 '01',
 '03',
 '01',
 '03',
 '05',
 '01',
 '03',
 '05',
 '01',
 '03',
 '05',
 '07',
 '09',
 '11',
 '13',
 '15',
 '17',
 '01',
 '03',
 '05',
 '07',
 '12',
 '01',
 '02',
 '03',
 '04',
 '05',
 '07',
 '11',
 '13',
 '15',
 '01',
 '03',
 '05',
 '07',
 '09',
 '01',
 '03',
 '05',
 '07',
 '09',
 '11',

In [129]:
medium = []
l_number = 1
for i in range(len(m_number)):
    if i == 0:
        medium.append('0101')
    else:
        if int(m_number[i]) < int(m_number[i-1]):
            l_number += 1
            medium.append(str(l_number)+m_number[i])
        else:
            medium.append(str(l_number)+m_number[i])

In [130]:
for i in range(len(medium)):
    if len(medium[i]) == 3:
        medium[i] = '0'+medium[i]

In [131]:
len(medium),len(medium_value)

(146, 146)

In [132]:
m_zip = zip(medium,medium_value)
medium_dict = dict(m_zip)
medium_dict

{'0101': '별, 혜성\n\n[Note]\n(a) 군인 계급을 표시하는 별을 포함\n(b) 스파크(01-15-07), 별 형상의 투척용 무기[닌자 스타](23-01-25), 다윗의 별(24-11-15), 별표(*)모양(24-17-03)은 포함하지 않음',
 '0103': '태양\n\n[Note]\n광선 없이 원형으로만 이루어진 태양(26-01)은 제외',
 '0105': '지구, 지구본, 행성',
 '0107': '달\n\n[Note]\n별과 함께 있는 달 포함',
 '0111': '성좌, 성군, 별이 있는 하늘, 천구의, 천체지도',
 '0113': '고대<아밀러리> 천구의, 천문관, 천문궤도, 원자 모형, 분자 모형',
 '0115': '자연현상\n\n[Note]\n소리 또는 전자기파를 연상하게 하는 선의 집합(26-11-22)은 포함하지 않음',
 '0117': '지리학적 지도, 평면 천체도(성좌도)\n\n[Note]\n두 개의 반구(01-05-01)로 이루어진 세계지도는 포함하지 않음',
 '0201': '남성',
 '0203': '여성',
 '0205': '어린이\n\n[Note]\n(02-01, 02-03 또는 02-07)로 분류되는 청년은 포함하지 않음',
 '0207': '혼성 그룹, 단체장면\n\n[Note]\n(a) 혼성그룹이란 남·여 그룹 또는 어른과 어린이 그룹을 말하며, 남성, 여성, 어린이 각각의 그룹은 (02-01), (02-03), (02-05)로 분류\n(b) 그러나 상기 어느 그룹이라도 본 분류상 특정 장면을 연상시키면 본 분류를 적용',
 '0209': '신체의 일부, 골격, 두개골',
 '0301': '네발짐승 (I 계열군)',
 '0302': '네발짐승 (II 계열군)',
 '0303': '네발짐승 (III 계열군)',
 '0304': '네발짐승 (IV 계열군)',
 '0305': '네발짐승 (V 계열군)',
 '0306': '네발짐승의 신체의 일부, 골격, 두개골',
 '0307': '조류, 박쥐',
 '0309':

### 소분류 딕트 pickle 파일 생성

In [133]:
soup.select('td.txt_left')

[<td class="txt_left">별</td>,
 <td class="txt_left">하나의 별</td>,
 <td class="txt_left">두 개의 별</td>,
 <td class="txt_left">세 개의 별</td>,
 <td class="txt_left">네 개 이상의 별
 
 [Note]
 중분류 (01-11)의 성좌 및 성군은 포함하지 않음</td>,
 <td class="txt_left">동일한 중심을 가진 별</td>,
 <td class="txt_left">삼각별</td>,
 <td class="txt_left">사각별</td>,
 <td class="txt_left">오각 이상의 별</td>,
 <td class="txt_left">꼭지점이 불균등한 별 
 
 [Note]
 꼭지점이 불균등한 별로 구성된 스파크를 포함</td>,
 <td class="txt_left">광선 또는 방사선을 가지는 별</td>,
 <td class="txt_left">불완전한(미완성의) 별</td>,
 <td class="txt_left">혜성, 유성</td>,
 <td class="txt_left">끝이 구형(구체)으로 된 별</td>,
 <td class="txt_left">나침반의 방위면(지침면)
 
 [Note]
 나침반·항해용 나침반(17-05-01)제외</td>,
 <td class="txt_left">사람의 얼굴 또는 동물의 머리를 표현하거나 의인화된 별</td>,
 <td class="txt_left">(01-11)로 분류되지 않는 별의 다른 표현</td>,
 <td class="txt_left">해돋이 또는 일몰</td>,
 <td class="txt_left">기타 다른 형태의 태양</td>,
 <td class="txt_left">풍경과 함께 있는 태양</td>,
 <td class="txt_left">사람 또는 사람 신체의 일부와 함께 있는 태양</td>,
 <td class="txt_left">동물과 함께 있는 태양</td>

In [134]:
vnumber = []
for i in range(len(soup.select('input')[7:])):
    vnumber.append(soup.select('input')[i+7]['value'])
del vnumber[-1] #마지막 값에 nan 들어있음

In [135]:
vtext = []
for i in range(len(soup.select('td.txt_left'))):
    vtext.append(soup.select('td.txt_left')[i].get_text())

In [137]:
len(vnumber),len(vtext)

(1837, 1837)

In [138]:
myzip = zip(vnumber,vtext)
vienna_dict = dict(myzip)
vienna_dict

{'010101': '별',
 '010102': '하나의 별',
 '010103': '두 개의 별',
 '010104': '세 개의 별',
 '010105': '네 개 이상의 별\n\n[Note]\n중분류 (01-11)의 성좌 및 성군은 포함하지 않음',
 '010106': '동일한 중심을 가진 별',
 '010108': '삼각별',
 '010109': '사각별',
 '010110': '오각 이상의 별',
 '010112': '꼭지점이 불균등한 별 \n\n[Note]\n꼭지점이 불균등한 별로 구성된 스파크를 포함',
 '010113': '광선 또는 방사선을 가지는 별',
 '010114': '불완전한(미완성의) 별',
 '010115': '혜성, 유성',
 '010116': '끝이 구형(구체)으로 된 별',
 '010117': '나침반의 방위면(지침면)\n\n[Note]\n나침반·항해용 나침반(17-05-01)제외',
 '010120': '사람의 얼굴 또는 동물의 머리를 표현하거나 의인화된 별',
 '010125': '(01-11)로 분류되지 않는 별의 다른 표현',
 '010301': '해돋이 또는 일몰',
 '010302': '기타 다른 형태의 태양',
 '010306': '풍경과 함께 있는 태양',
 '010307': '사람 또는 사람 신체의 일부와 함께 있는 태양',
 '010308': '동물과 함께 있는 태양',
 '010309': '식물과 함께 있는 태양',
 '010310': '구름, 비, 물방울, 또는 기타 다른 자연현상과 함께 있는 태양',
 '010311': '제조 물품 또는 산업용 제품과 함께 도시된 태양',
 '010312': '기타 다른 도형 요소와 함께 도시된 태양',
 '010313': '문자와 함께 있는 태양(포함한 경우)',
 '010315': '직선(선 다발) 또는 띠(띠 묶음)로 구성되는 광선을 가지는 태양',
 '010316': '불꽃모양·물결모양의 선(선 다발) 또는 띠(띠 묶음)로 구성되는 광선을 가지는 태양',
 '01

# 대분류,중분류,소분류 Dict 통합 파일 생성

In [143]:
main_category.update(medium_dict)
main_category.update(vienna_dict)
len(main_category)

2013

In [145]:
with open('vienna_dict.pickle','wb') as f:
    pickle.dump(main_category, f, pickle.HIGHEST_PROTOCOL)